In [73]:
import numpy as np
import matplotlib.pyplot as plt

import time

import scipy.signal
from gym.spaces import Box, Discrete
from torch.autograd import Variable

import torch
from torch.optim import Adam
import torch.nn as nn
from torch.distributions.categorical import Categorical

In [74]:
def discount_cumsum(x, discount):
    """
    Compute  cumulative sums of vectors.

    Input: [x0, x1, ..., xn]
    Output: [x0 + discount * x1 + discount^2 * x2, x1 + discount * x2, ..., xn]
    """
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]

def combined_shape(length, shape=None):
    """Helper function that combines two array shapes."""
    if shape is None:
        return (length,)
    return (length, shape) if np.isscalar(shape) else (length, *shape)

def mlp(sizes, activation, output_activation=nn.Identity):
    """The basic multilayer perceptron architecture used."""
    layers = []
    for j in range(len(sizes)-1):
        act = activation if j < len(sizes)-2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
    return nn.Sequential(*layers)

In [75]:
class MLPCategoricalActor(nn.Module):
    """A class for the policy network."""

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation):
        super().__init__()
        self.logits_net = mlp([obs_dim] + list(hidden_sizes) + [act_dim], activation)

    def _distribution(self, obs):
        """Takes the observation and outputs a distribution over actions."""
        logits = self.logits_net(obs)
        return Categorical(logits=logits)

    def _log_prob_from_distribution(self, pi, act):
        """
        Take a distribution and action, then gives the log-probability of the action
        under that distribution.
        """
        return pi.log_prob(act)

    def forward(self, obs, act=None):
        """
        Produce action distributions for given observations, and then compute the
        log-likelihood of given actions under those distributions.
        """
        pi = self._distribution(obs)
        logp_a = None
        if act is not None:
            logp_a = self._log_prob_from_distribution(pi, act)
        return pi, logp_a
    
    #Filip Add method
    def sample(self, obs):
        
        pi = self._distribution(obs)
        action = pi.sample()
        logp_a = self._log_prob_from_distribution(pi, action)
        return action, logp_a

class MLPCritic(nn.Module):
    """The network used by the value function."""
    def __init__(self, obs_dim, hidden_sizes, activation):
        super().__init__()
        self.v_net = mlp([obs_dim] + list(hidden_sizes) + [1], activation)

    def forward(self, obs):
        # Critical to ensure v has right shape
        return torch.squeeze(self.v_net(obs), -1)


class MLPActorCritic(nn.Module):
    """Class to combine policy and value function neural networks."""

    def __init__(self,
                 hidden_sizes=(64,64), activation=nn.Tanh):
        super().__init__()

        obs_dim = 8

        # Build policy for 4-dimensional action space
        self.pi = MLPCategoricalActor(obs_dim, 4, hidden_sizes, activation)

        # Build value function
        self.v  = MLPCritic(obs_dim, hidden_sizes, activation)

    def step(self, state):
        """
        Take an state and return action, value function, and log-likelihood
        of chosen action.
        """
        # TODO: Implement this function.
        # It is supposed to return three numbers:
        #    1. An action sampled from the policy given a state (0, 1, 2 or 3)
        #    2. The value function at the given state
        #    3. The log-probability of the action under the policy output distribution
        # Hint: This function is only called during inference. You should use
        # `torch.no_grad` to ensure that it does not interfer with the gradient computation.
        
        with torch.no_grad():
            action, logp_action = self.pi.sample(state)
            val_f = self.v.forward(state)
        
        return action.item(), val_f.item(), logp_action.item()

    def act(self, state):
        return self.step(state)[0]

In [76]:
class VPGBuffer:
    """
    Buffer to store trajectories.
    """
    def __init__(self, obs_dim, act_dim, size, gamma, lam):
        self.obs_buf = np.zeros(combined_shape(size, obs_dim), dtype=np.float32)
        self.act_buf = np.zeros(combined_shape(size, act_dim), dtype=np.float32)
        # calculated TD residuals
        self.tdres_buf = np.zeros(size, dtype=np.float32)
        # rewards
        self.rew_buf = np.zeros(size, dtype=np.float32)
        # trajectory's remaining return
        self.ret_buf = np.zeros(size, dtype=np.float32)
        # values predicted
        self.val_buf = np.zeros(size, dtype=np.float32)
        # log probabilities of chosen actions under behavior policy
        self.logp_buf = np.zeros(size, dtype=np.float32)
        self.gamma = gamma
        self.lam = lam
        self.ptr, self.path_start_idx, self.max_size = 0, 0, size

    def store(self, obs, act, rew, val, logp):
        """
        Append a single timestep to the buffer. This is called at each environment
        update to store the outcome observed outcome.
        """
        # buffer has to have room so you can store
        assert self.ptr < self.max_size
        self.obs_buf[self.ptr] = obs
        self.act_buf[self.ptr] = act
        self.rew_buf[self.ptr] = rew
        self.val_buf[self.ptr] = val
        self.logp_buf[self.ptr] = logp
        self.ptr += 1

    def end_traj(self, last_val=0):
        """
        Call after a trajectory ends. Last value is value(state) if cut-off at a
        certain state, or 0 if trajectory ended uninterrupted
        """
        path_slice = slice(self.path_start_idx, self.ptr)
        rews = np.append(self.rew_buf[path_slice], last_val)
        vals = np.append(self.val_buf[path_slice], last_val)

        # TODO: Implement TD residual calculation.
        # Hint: we do the discounting for you, you just need to compute 'deltas'.
        # see the handout for more info
        # deltas = rews[:-1] + ...
        
        deltas = rews[:-1] + vals[1::] - vals[:-1] # can also average val across episodes
        self.tdres_buf[path_slice] = discount_cumsum(deltas, self.gamma*self.lam)

        #TODO: compute the discounted rewards-to-go. Hint: use the discount_cumsum function
        
        self.ret_buf[path_slice] = discount_cumsum(rews[:-1], self.gamma)
        self.path_start_idx = self.ptr


    def get(self):
        """
        Call after an epoch ends. Resets pointers and returns the buffer contents.
        """
        # Buffer has to be full before you can get something from it.
        assert self.ptr == self.max_size
        self.ptr, self.path_start_idx = 0, 0

        # TODO: Normalize the TD-residuals in self.tdres_buf
        self.tdres_buf = (self.tdres_buf - np.mean(self.tdres_buf)) / np.std(self.tdres_buf)

        data = dict(obs=self.obs_buf, act=self.act_buf, ret=self.ret_buf, tdres=self.tdres_buf, logp=self.logp_buf)
        return {k: torch.as_tensor(v, dtype=torch.float32) for k,v in data.items()}

In [94]:
class Agent:
    def __init__(self, env):
        self.env = env
        self.hid = 64  # layer width of networks
        self.l = 2  # layer number of networks
        # initialises an actor critic
        self.ac = MLPActorCritic(hidden_sizes=[self.hid]*self.l)

    def train(self):
        """
        Main training loop.

        IMPORTANT: This function called by the checker to train your agent.
        You SHOULD NOT change the arguments this function takes and what it outputs!
        """

        # The observations are 8 dimensional vectors, and the actions are numbers,
        # i.e. 0-dimensional vectors (hence act_dim is an empty list).
        obs_dim = [8]
        act_dim = []

        # Training parameters
        # You may wish to change the following settings for the buffer and training
        # Number of training steps per epoch
        steps_per_epoch = 3000
        # Number of epochs to train for
        epochs = 50
        # The longest an episode can go on before cutting it off
        max_ep_len = 300
        # Discount factor for weighting future rewards
        gamma = 0.99
        lam = 0.97

        # Learning rates for policy and value function
        pi_lr = 3e-3
        vf_lr = 1e-3

        # Set up buffer
        buf = VPGBuffer(obs_dim, act_dim, steps_per_epoch, gamma, lam)

        # Initialize the ADAM optimizer using the parameters
        # of the policy and then value networks
        # TODO: Use these optimizers later to update the policy and value networks.
        pi_optimizer = Adam(self.ac.pi.parameters(), lr=pi_lr)
        v_optimizer = Adam(self.ac.v.parameters(), lr=vf_lr)

        # Initialize the environment
        state, ep_ret, ep_len = self.env.reset(), 0, 0
        
        ret_sums = 0 #Filip
        mse_loss = nn.MSELoss() #Filip

        # Main training loop: collect experience in env and update / log each epoch
        for epoch in range(epochs):
            ep_returns = []
            list_logp_a_grad = [] # Filip
            list_v_grad = [] # Filip
            
            for t in range(steps_per_epoch):
                
                a, v, logp = self.ac.step(torch.as_tensor(state, dtype=torch.float32))
                
                pi, logp_a_grad = self.ac.pi.forward(torch.as_tensor(state, dtype=torch.float32), torch.tensor(a)) #Filip
                list_logp_a_grad.append(logp_a_grad) #Filip
                
                v_grad = self.ac.v.forward(torch.as_tensor(state, dtype=torch.float32)) #Filip
                list_v_grad.append(v_grad) #Filip
                
                next_state, r, terminal = self.env.transition(a)
                ep_ret += r
                ep_len += 1

                # Log transition
                buf.store(state, a, r, v, logp)

                # Update state (critical!)
                state = next_state

                timeout = ep_len == max_ep_len
                epoch_ended = (t == steps_per_epoch - 1)

                if terminal or timeout or epoch_ended:
                    # if trajectory didn't reach terminal state, bootstrap value target
                    if epoch_ended:
                        _, v, _ = self.ac.step(torch.as_tensor(state, dtype=torch.float32))
                    else:
                        v = 0
                    if timeout or terminal:
                        ep_returns.append(ep_ret)  # only store return when episode ended
                    buf.end_traj(v)
                    state, ep_ret, ep_len = self.env.reset(), 0, 0

            mean_return = np.mean(ep_returns) if len(ep_returns) > 0 else np.nan
            print(f"Epoch: {epoch+1}/{epochs}, mean return {mean_return}")

            # This is the end of an epoch, so here is where you likely want to update
            # the policy and / or value function.
            # TODO: Implement the policy and value function update. Hint: some of the torch code is
            # done for you.
            
            data = buf.get()
            loss_pi = None
            if epoch == 0:
                sum_tdres = data['tdres']
            else:
                sum_tdres += data['tdres']
                
            tensor_logp_a_grad = torch.tensor(list_logp_a_grad) # dif between tensor and Tensor?
            tensor_v_grad = torch.tensor(list_v_grad)
            
            tensor_logp_a_grad.requires_grad = True
            tensor_v_grad.requires_grad = True

            pi_optimizer.zero_grad()
            if loss_pi is None:
                loss_pi = torch.sum(torch.mul(tensor_logp_a_grad, data['tdres']))
            else:
                loss_pi += torch.sum(torch.mul(tensor_logp_a_grad, data['tdres']))
            
            loss_pi_new = loss_pi.mul(-1/((epoch+1)*t))
            #print("Tensor of log probabilities", tensor_logp_a_grad)
            #print("Tensor of sum of residuals", sum_tdres)
            #print("Tensor after multiplication", )
            print("Tensor after sum", loss_pi)
            print("-----------")
            
            loss_pi_new.backward()
            pi_optimizer.step()
            
            for _ in range(100):
                
                v_optimizer.zero_grad()
                loss_v = mse_loss(tensor_v_grad, data['ret'])
                loss_v.backward()
                v_optimizer.step()


        return True


    def get_action(self, obs):
        """
        Sample an action from your policy.

        IMPORTANT: This function called by the checker to evaluate your agent.
        You SHOULD NOT change the arguments this function takes and what it outputs!
        """
        # TODO: Implement this function.

        return self.ac.act(obs)


def main():
    """
    Train and evaluate agent.

    This function basically does the same as the checker that evaluates your agent.
    You can use it for debugging your agent and visualizing what it does.
    """
    from lunar_lander import LunarLander
    from gym.wrappers.monitoring.video_recorder import VideoRecorder

    env = LunarLander()

    agent = Agent(env)
    agent.train()

    rec = VideoRecorder(env, "policy.mp4")
    episode_length = 300
    n_eval = 100
    returns = []
    print("Evaluating agent...")

    for i in range(n_eval):
        print(f"Testing policy: episode {i+1}/{n_eval}")
        state = env.reset()
        cumulative_return = 0
        # The environment will set terminal to True if an episode is done.
        terminal = False
        env.reset()
        for t in range(episode_length):
            if i <= 10:
                rec.capture_frame()
            # Taking an action in the environment
            action = agent.get_action(state)
            state, reward, terminal = env.transition(action)
            cumulative_return += reward
            if terminal:
                break
        returns.append(cumulative_return)
        print(f"Achieved {cumulative_return:.2f} return.")
        if i == 10:
            rec.close()
            print("Saved video of 10 episodes to 'policy.mp4'.")
    env.close()
    print(f"Average return: {np.mean(returns):.2f}")

if __name__ == "__main__":
    main()


Epoch: 1/50, mean return -291.4248993164757
Tensor after sum tensor(-1.0382, grad_fn=<SumBackward0>)
-----------
Epoch: 2/50, mean return -349.710548484882
Tensor after sum tensor(14.2426, grad_fn=<SumBackward0>)
-----------
Epoch: 3/50, mean return -364.4627727514372
Tensor after sum tensor(17.4178, grad_fn=<SumBackward0>)
-----------
Epoch: 4/50, mean return -350.7280488025321
Tensor after sum tensor(2.0790, grad_fn=<SumBackward0>)
-----------
Epoch: 5/50, mean return -316.95959686615237
Tensor after sum tensor(0.8060, grad_fn=<SumBackward0>)
-----------
Epoch: 6/50, mean return -294.3592022523435
Tensor after sum tensor(27.6806, grad_fn=<SumBackward0>)
-----------
Epoch: 7/50, mean return -331.7494288483622
Tensor after sum tensor(-2.5947, grad_fn=<SumBackward0>)
-----------
Epoch: 8/50, mean return -296.22113064004924
Tensor after sum tensor(-5.9317, grad_fn=<SumBackward0>)
-----------
Epoch: 9/50, mean return -355.39982955923074
Tensor after sum tensor(11.9605, grad_fn=<SumBackwar

KeyboardInterrupt: 

In [66]:
a = [torch.tensor(1), torch.tensor(2)]
x = Variable(torch.tensor(a), requires_grad = True)

a = torch.tensor([1,2,3])
b = torch.tensor([4,5,6])
(a+b) / 2.0

tensor([2.5000, 3.5000, 4.5000])

In [83]:
A = np.array([1,2,3])
discount_cumsum(A, 1)

array([6., 5., 3.])